<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [19]:
import azureml.core
from azureml.data.data_reference import DataReference
from azureml.data.datapath import DataPath
from azureml.core import Workspace, Datastore, Dataset
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

In [21]:
ws = Workspace.from_config()
def_blob_store = Datastore(ws, "workspaceblobstore")
steps_dir = './pipeline_steps'
cpu_cluster_name = "cpucluster"

In [24]:
cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
cpu_cluster.wait_for_completion(show_output=True)

# Create a new runconfig object
run_amlcompute = RunConfiguration()

# Use the cpu_cluster you created above. 
run_amlcompute.target = cpu_cluster

# Enable Docker
run_amlcompute.environment.docker.enabled = True

# Set Docker base image to the default CPU-based image
run_amlcompute.environment.docker.base_image = DEFAULT_CPU_IMAGE

# Use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_amlcompute.environment.python.user_managed_dependencies = False

# Specify CondaDependencies obj, add necessary packages
#run_amlcompute.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [27]:
dataset_full = Dataset.get_by_name(ws, name="annonces_ds")

clean_ds = PipelineData("dataset_clean",
                        datastore=def_blob_store)

clean_step = PythonScriptStep(
    script_name="clean.py",
    arguments=["--input", dataset_full.name, "--output", clean_ds.name],
    inputs=[dataset_full.as_named_input('annonce_ds')],
    outputs=[clean_ds],
    compute_target=cpu_cluster,
    source_directory=steps_dir
)

In [28]:
train_pipeline = Pipeline(workspace=ws, steps=[clean_step])

In [35]:
dataset_clean = clean_ds.as_dataset()
#dataset_clean.register("fulltrain")
#clean_ds = Dataset.get_by_name(ws, name="fulltrain")
clean_df = dataset_clean.to_pandas_dataframe()

clean_df.head()

AttributeError: 'PipelineOutputFileDataset' object has no attribute 'to_pandas_dataframe'

In [36]:
help(clean_ds)

Help on PipelineData in module azureml.pipeline.core.builder object:

class PipelineData(builtins.object)
 |  PipelineData(name, datastore=None, output_name=None, output_mode='mount', output_path_on_compute=None, output_overwrite=None, data_type=None, is_directory=None, pipeline_output_name=None, training_output=None)
 |  
 |  Represents intermediate data in an Azure Machine Learning pipeline.
 |  
 |  Data used in pipeline can be produced by one step and consumed in another step by providing a PipelineData
 |  object as an output of one step and an input of one or more subsequent steps.
 |  
 |  .. remarks::
 |  
 |      PipelineData represents data output a step will produce when it is run. Use PipelineData when creating steps
 |      to describe the files or directories which will be generated by the step. These data outputs will be added to
 |      the specified Datastore and can be retrieved and viewed later.
 |  
 |      For example, the following pipeline step produces one outpu